# Выбор локации для скважины

Добывающей природные ресурсы компании нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных
Данные геологоразведки трёх регионов находятся в файлах:  
geo_data_0.csv.  
geo_data_1.csv.  
geo_data_2.csv.  
id — уникальный идентификатор скважины;  
#### признаки
f0, f1, f2  
#### целевой признак
product — объём запасов в скважине (тыс. баррелей).  

# 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

Открываем данные о трех регионах

In [2]:
# регион 0
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_0.info()
geo_data_0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
# регион 1
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_1.info()
geo_data_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
# регион 2
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
geo_data_2.info()
geo_data_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Рассмотрим минимумы и максимумы признаков, чтобы выяснить, есть ли выбросы и надо ли масштабировать признаки

In [5]:
for data in [geo_data_0, geo_data_1, geo_data_2]:
    print('максимумы:\n', data.max())
    print('минимумы:\n', data.min())

максимумы:
 id           zzzLH
f0         2.36233
f1         1.34377
f2         16.0038
product    185.364
dtype: object
минимумы:
 id            006OJ
f0         -1.40861
f1        -0.848218
f2         -12.0883
product           0
dtype: object
максимумы:
 id           zzzvI
f0         29.4218
f1         18.7341
f2         5.01972
product    137.945
dtype: object
минимумы:
 id             0022J
f0          -31.6096
f1          -26.3586
f2        -0.0181441
product            0
dtype: object
максимумы:
 id           zzz9h
f0         7.23826
f1          7.8448
f2         16.7394
product     190.03
dtype: object
минимумы:
 id           009Gl
f0           -8.76
f1        -7.08402
f2        -11.9703
product          0
dtype: object


#### Вывод

* Данные пропусков не имеют 
* Приводить к другим типам не надо 
* Масштабирование признаков проводить не надо
* Столбцы **id** использоваться при построении моделей не будут, так как не несут информации о запасах сырья

# 2. Обучение и проверка модели

Разделяем данные на тренировочную и валидационные выборки

In [6]:
# определяем функцию для разделения на выборки
def geo_split(data):
    target = data['product']
    features = data.drop(['id', 'product'], axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid

In [7]:
# разделяем данные на выборки
features_train_0, features_valid_0, target_train_0, target_valid_0 = geo_split(geo_data_0)
features_train_1, features_valid_1, target_train_1, target_valid_1 = geo_split(geo_data_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = geo_split(geo_data_2)

Обучаем модели линеной регрессии

In [8]:
def geo_prediction(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    print('Средний запас предсказаного сырья = {:0.2f}'.format(predicted_valid.mean()))
    print('Стандартное отклонение = {:0.2f}'.format(rmse))
    return pd.Series(data = predicted_valid, index = target_valid.index), target_valid

In [9]:
print('Регион 0')
predicted_valid_0, target_valid_0 = geo_prediction(features_train_0, features_valid_0, target_train_0, target_valid_0)

Регион 0
Средний запас предсказаного сырья = 92.59
Стандартное отклонение = 37.58


In [10]:
print('Регион 1')
predicted_valid_1, target_valid_1 = geo_prediction(features_train_1, features_valid_1, target_train_1, target_valid_1)

Регион 1
Средний запас предсказаного сырья = 68.73
Стандартное отклонение = 0.89


In [11]:
print('Регион 2')
predicted_valid_2, target_valid_2 = geo_prediction(features_train_2, features_valid_2, target_train_2, target_valid_2)

Регион 2
Средний запас предсказаного сырья = 94.97
Стандартное отклонение = 40.03


#### Вывод

В регионах 0 и 2 стандартное отклонение составляет почти 40% от среднего запаса предсказанного сырья, а это значит, что имеющихся признаков недостаточно для однозначного определения нужных скважин. В регионе 1 модель, конечно, довольно точно предсказывает запасы сырья, но средние запасы в этом регионе почти на четверть меньше, чем в остальных. Стоит посчитат риски, вдруг в 0 или 2 регионе будет бурить выгоднее, чем в регионе 1.  

# 3. Подготовка к расчёту прибыли

Запишем ключевые для расчета значения 

In [12]:
# общий бюджет
BUDGET = 10000000000
# доход с тысячи баррелей
INCOME = 450000
# точек для исследования
POINTS = 500
# точек для бурения
BEST_POINTS = 200
# максимальная вероятность убытков
MAX_RISK = 0.025

Рассчитаем, сколько должно быть в среднем в 200 скважинах сырья, чтобы разарботка была безубыточной:

In [13]:
V_BREAKEVEN = BUDGET / BEST_POINTS / INCOME
print("Необходимый средний запас сырья, тыс. баррелей = {:0.2f}".format(V_BREAKEVEN))

Необходимый средний запас сырья, тыс. баррелей = 111.11


#### Вывод

Необходимый для безубыточности средний запас сырья 200 скважин больше, чем средний запас сырья в скважине в любом регионе

# 4. Расчёт прибыли и рисков 

Расчет будем проводить из следующих соображений: исследуется 500 возможных скважин и из них выбираются 200 с наибольшими запасами сырья.

Напишем функцию для расчета прибыли

In [94]:
# функция для рассчета прибыли
# выбирает 200 лучших скважин
# суммирует залежи и умнажает на INCOME
# и вычитает бюджет из полученного
def revenue(target_valid, predicted_valid):
    predicted_valid_sorted = predicted_valid.sort_values(ascending=False)
    target_valid_selected = target_valid[predicted_valid_sorted.index][:BEST_POINTS]
    return INCOME * target_valid_selected.sum() - BUDGET

Занесем в один список целевые признаки и предсказанные значения для всех регионов

In [15]:
list_target_predicted = [target_valid_0, predicted_valid_0, target_valid_1, predicted_valid_1, target_valid_2, predicted_valid_2]

Проведем процедуру Bootstrap, то есть для каждого региона 1000 раз выберем наугад 500 скважин и посчитаем прибыль, 95% доверительных интервал и риск убытков.

In [108]:
state = np.random.RandomState(12345)   
for valid in range(0, len(list_target_predicted), 2):
    values = []
    for i in range(1000):
        predicted_valid_subsample = list_target_predicted[valid + 1].sample(n=POINTS, replace=True, random_state=state)
        target_valid_subsample = list_target_predicted[valid][predicted_valid_subsample.index]
        values.append(revenue(target_valid_subsample, predicted_valid_subsample))
    values = pd.DataFrame(data = values)
    lower = values[0].quantile(MAX_RISK)
    higher = values[0].quantile(0.975)
    mean = values[0].mean()
    print('Регион', int(valid / 2))
    print("Средняя выручка: {:0.2f}".format(mean))
    print("2,5%-квантиль: {:0.2f}".format(lower))
    print('97,5%-квантиль: {:0.2f}'.format(higher) ) 
    print('95% доверительный интервал: от {:0.2f} до {:0.2f}'.format(lower, higher))
    print('Риск убытков: {:0.1%}\n'.format(values[values[0] < 0][0].count() / values[0].count()))

Регион 0
Средняя выручка: 425938526.91
2,5%-квантиль: -102090094.84
97,5%-квантиль: 947976353.36
95% доверительный интервал: от -102090094.84 до 947976353.36
Риск убытка: 6.0%

Регион 1
Средняя выручка: 518259493.70
2,5%-квантиль: 128123231.43
97,5%-квантиль: 953612982.07
95% доверительный интервал: от 128123231.43 до 953612982.07
Риск убытка: 0.3%

Регион 2
Средняя выручка: 420194005.34
2,5%-квантиль: -115852609.16
97,5%-квантиль: 989629939.84
95% доверительный интервал: от -115852609.16 до 989629939.84
Риск убытка: 6.2%



# Вывод

Наибольшая средняя прибыль (500 млн.) предсказана для региона 1, несмотря на то, что модель предсказывает там наименьшие средние запасы сырья в скважинах среди всех. Кроме того, регион 1 - единственный регион, у которого риск убытков меньше 2.5%, а именно 0.6%. Поэтому лучшим регионом для бурения является регион 1.